## Evaluation 

### Inhalt

Beschreibung des Quellcodes der Basis-Evaluation. 

Der Quellcode ist funktional gegliedert. Das heißt in einer Zelle wird eine inhaltlich separierbare Funktion ausgeführt.  

**Schritt 1**
Im ersten Schritt werden die relevanten Bibliotheken für die Evaluation eingebunden

In [1]:

import os, numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import load_model

**Schritt 2**
Im zweiten Schritt werden die relevanten Pfade zu den benötigten Dateien angelegt:
- Datensatz german.data
- Künstliches Neuronales Netz german_credit_model.keras
- Diverse Profile aus dem German Credit Datensatz diverse_profiles.csv
- Profilbericht über die Ergebnisse der beiden Metriken. Dieser wird durch die Evaluation generiert und als profilbericht_alpha_beta_twostage.md abgespeichert. 

In [9]:

# --- Pfade ---
DATA_PATH = r"C:\Users\JonasNiehus\Documents\Masterarbeit\Evaluation\Masterarbeit_Guetekriterien-sichere-und-interpetierbare-Hochrisiko-KI-Systeme\data\german.data"
MODEL_PATH = r"C:\Users\JonasNiehus\Documents\Masterarbeit\Evaluation\Masterarbeit_Guetekriterien-sichere-und-interpetierbare-Hochrisiko-KI-Systeme\models\german_credit_model.keras"
PROFILES_CSV = r"C:\Users\JonasNiehus\Documents\Masterarbeit\Evaluation\Masterarbeit_Guetekriterien-sichere-und-interpetierbare-Hochrisiko-KI-Systeme\results\diverse_profiles.csv"
REPORT_PATH = r"C:\Users\JonasNiehus\Documents\Masterarbeit\Evaluation\Masterarbeit_Guetekriterien-sichere-und-interpetierbare-Hochrisiko-KI-Systeme\results\profilbericht_alpha_beta_twostage.md"

**Schritt 3**
Hier wird über DATA_PATH der German-Credit-Datensatz eingelesen und die Struktur definiert:
- Spaltennamen werden vergeben 
- Die Zielvariable wird auf binär gemappt {1→1 (good), 2→0 (bad)}.


In [10]:
# Laden & Vorverarbeitung
df = pd.read_csv(DATA_PATH, header=None, sep=r"\s+")
df.columns = [
    "Status_des_Girokontos", "Dauer_in_Monaten", "Kreditgeschichte", "Kreditverwendungszweck",
    "Kreditbetrag", "Sparkonto_Wertpapiere", "Beschäftigt_seit", "Ratenhöhe",
    "Familienstand_Geschlecht", "Weitere_Bürgen_Schuldner", "Wohnsitzdauer", "Vermögen", "Alter",
    "Andere_Ratenverpflichtungen", "Wohnsituation", "Anzahl_bestehender_Kredite", "Beruf",
    "Unterhaltspflichtige_Personen", "Telefon", "Ausländischer_Arbeiter", "Ziel"
]
df["Ziel"] = df["Ziel"].map({1: 1, 2: 0}).astype(int)

X_all = df.drop(columns=["Ziel"])
y_all = df["Ziel"].values

**Schritt 4** Hier werden die kategorischen und numerischen Variablen behandelt 
- Mit numerical_cols und categorical_cols wird definiert, welche Spalten numerisch und welche kategorisch behandelt werden.

- Der ColumnTransformer standardisiert numerische Spalten und kategoriale werden one-hot encodiert.

In [11]:
# Numerisch/Kategorisch
numerical_cols = [
    "Dauer_in_Monaten", "Kreditbetrag", "Ratenhöhe", "Wohnsitzdauer",
    "Alter", "Anzahl_bestehender_Kredite", "Unterhaltspflichtige_Personen"
]
categorical_cols = [c for c in X_all.columns if c not in numerical_cols]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
])
preprocessor.fit(X_all)


,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


**Schritt 5** Hier wird das ANN für die Kreditprognosen geladen

- Die Bedingung prüft, ob das Modell im Pfad existiert 
- Dann wird das Modell mit model geladen


In [12]:
# --- Modell laden ---
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Modelldatei fehlt: {MODEL_PATH}")
model = load_model(MODEL_PATH)
print("Modell erfolgreich geladen.")

Modell erfolgreich geladen.


**Schritt 6** Einbindung der 10 diversen Profile 

- Die 10 Profile werden mittels profiles_df eingelesen
- Es werden überflüssige Spalten entfernt 
- Die Spalten werden geordnet wie in X_all.

In [13]:

# --- Profile laden & Spalten ausrichten ---
profiles_df = pd.read_csv(PROFILES_CSV)
extra_cols = [c for c in profiles_df.columns if c not in X_all.columns]
if extra_cols:
    profiles_df = profiles_df.drop(columns=extra_cols)
profiles_df = profiles_df[X_all.columns]
profiles_df[numerical_cols] = profiles_df[numerical_cols].apply(pd.to_numeric, errors="coerce").astype(float)


**Schritt 7** Ökonomishce und Soziodemographische Features

- Die ökonomischen und soziodemographischen Feature-Gruppen werden mit _num und _cat explizit definiert


In [14]:
# Feste Feature-Gruppen 
econ_num = ['Kreditbetrag', 'Dauer_in_Monaten', 'Ratenhöhe', 'Anzahl_bestehender_Kredite']
econ_cat = ['Kreditgeschichte', 'Kreditverwendungszweck', 'Sparkonto_Wertpapiere', 'Vermögen',
            'Status_des_Girokontos', 'Beschäftigt_seit', 'Andere_Ratenverpflichtungen', 'Weitere_Bürgen_Schuldner']

socio_num = ['Alter', 'Wohnsitzdauer', 'Unterhaltspflichtige_Personen']
socio_cat = ['Familienstand_Geschlecht', 'Wohnsituation', 'Telefon', 'Ausländischer_Arbeiter', 'Beruf']


**Schritt 8** Berechnung der beiden Metriken 

*α (Notwendigkeit, lokal):*

Für jedes Profil & Feature: 
Jedes Featrue wird isoliert variiert (numerisch über ein Quantil-Grid; kategorial über andere Klassen) → sobald der Output kippt, gilt das Feature als „notwendig“ (1), sonst 0.

*β (Suffizienz, lokal):*
Für jedes Profil & Feature: 
Das Feature bleibt stabild und es werden alle anderen Features variiert (durch Sampling aus X_all) → β ist der Anteil der Fälle, in denen der Output gleich bleibt wie beim Profil.


In [17]:
# α / β – Funktionen
def _predict_label_and_proba(model, preprocessor, row_df, threshold=0.5):
    Xp = preprocessor.transform(row_df)
    proba = float(model.predict(Xp, verbose=0)[0, 0])
    label = int(proba >= threshold)
    return label, proba

def _numeric_grid_around(value, lo, hi, n_steps=9):
    grid = np.linspace(lo, hi, n_steps)
    return [v for v in grid if abs(v - value) > 1e-12]

def alpha_test_numeric(model, preprocessor, df_row, feature, bounds, threshold=0.5,
                       n_steps=9, clip_min=None):
    orig_label, _ = _predict_label_and_proba(model, preprocessor, df_row, threshold)
    lo, hi = bounds
    value = float(df_row[feature].iloc[0])
    test_values = _numeric_grid_around(value, lo, hi, n_steps=n_steps)

    for v in test_values:
        v_clip = float(max(v, clip_min)) if clip_min is not None else float(v)
        mod_row = df_row.copy()
        mod_row.at[df_row.index[0], feature] = np.float64(v_clip)
        new_label, _ = _predict_label_and_proba(model, preprocessor, mod_row, threshold)
        if new_label != orig_label:
            return 1
    return 0

def alpha_test_categorical(model, preprocessor, df_row, feature, all_categories, threshold=0.5):
    orig_label, _ = _predict_label_and_proba(model, preprocessor, df_row, threshold)
    current_cat = df_row[feature].iloc[0]
    for cat in all_categories:
        if cat == current_cat:
            continue
        mod_row = df_row.copy()
        mod_row.at[df_row.index[0], feature] = cat
        new_label, _ = _predict_label_and_proba(model, preprocessor, mod_row, threshold)
        if new_label != orig_label:
            return 1
    return 0

def run_alpha_for_profiles(
    model, preprocessor, df_all, profiles_df,
    features_numeric, features_categorical,
    threshold=0.5, q_lo=0.05, q_hi=0.95, n_steps=9
):
    if not features_numeric and not features_categorical:
        return pd.DataFrame(index=profiles_df.index), pd.Series(dtype=float)

    num_bounds = {feat: (float(df_all[feat].quantile(q_lo)),
                         float(df_all[feat].quantile(q_hi)))
                  for feat in features_numeric}
    cat_values = {feat: sorted(df_all[feat].dropna().unique().tolist())
                  for feat in features_categorical}

    rows = []
    for i in profiles_df.index:
        row = profiles_df.loc[[i]]
        row_result = {}
        for feat in features_numeric:
            row_result[feat] = alpha_test_numeric(model, preprocessor, row, feature=feat,
                                                  bounds=num_bounds[feat], threshold=threshold,
                                                  n_steps=n_steps, clip_min=0.0)
        for feat in features_categorical:
            row_result[feat] = alpha_test_categorical(model, preprocessor, row, feature=feat,
                                                      all_categories=cat_values[feat], threshold=threshold)
        row_result["profile_index"] = i
        rows.append(row_result)

    results_table = pd.DataFrame(rows).set_index("profile_index").sort_index()
    alpha_per_feature = results_table.mean(axis=0).rename("alpha_rate")
    return results_table, alpha_per_feature

def _predict_label_and_proba_batch(model, preprocessor, df, threshold=0.5):
    Xp = preprocessor.transform(df)
    proba = model.predict(Xp, verbose=0).reshape(-1)
    labels = (proba >= threshold).astype(int)
    return labels, proba

def beta_test_feature(
    model, preprocessor, df_all, df_row, feature, value=None,
    threshold=0.5, n_samples=2000, random_state=42
):
    rng = np.random.default_rng(random_state)
    y_star, p_star = _predict_label_and_proba(model, preprocessor, df_row, threshold)
    a = df_row[feature].iloc[0] if value is None else value

    sample_idx = rng.integers(0, df_all.shape[0], size=n_samples)
    Z = df_all.iloc[sample_idx].copy()
    if pd.api.types.is_numeric_dtype(df_all[feature]):
        Z[feature] = float(a)
    else:
        Z[feature] = a

    y_hat, _ = _predict_label_and_proba_batch(model, preprocessor, Z, threshold)
    same = (y_hat == y_star).astype(float)
    beta = float(same.mean())

    return {"feature": feature, "a": a, "beta": beta, "y_star": int(y_star), "proba_star": float(p_star)}

def run_beta_for_profiles(
    model, preprocessor, df_all, profiles_df, features_numeric, features_categorical,
    threshold=0.5, n_samples=2000, random_state=42
):
    features = (features_numeric or []) + (features_categorical or [])
    if not features:
        return pd.DataFrame(index=profiles_df.index), pd.Series(dtype=float)

    rows = []
    for i in profiles_df.index:
        row = profiles_df.loc[[i]]
        row_result = {}
        for feat in features:
            res = beta_test_feature(
                model, preprocessor, df_all, row, feat,
                value=None, threshold=threshold,
                n_samples=n_samples, random_state=random_state + i,
            )
            row_result[feat] = res["beta"]
        row_result["profile_index"] = i
        rows.append(row_result)

    beta_table = pd.DataFrame(rows).set_index("profile_index").sort_index()
    beta_mean = beta_table.mean(axis=0)
    return beta_table, beta_mean

**Schritt 9** Erstellung des Profilsberichts

Die Funktion def build_two_stage_profile_report_md baut einen lesbaren Markdown-Bericht:

- Meta-Daten (Anzahl Profile, Schwelle β≥…)
- Übersichts-Tabelle je Profil (Anzahl notwendiger / hinreichender Features pro Stufe)
- Eine Detailtabellen je Profil (α=0/1, β-Wert, Indikator hinreichend)
- Das ganze wird unter REPORT_PATH gespeichert 

In [20]:
# Report-Builder (Markdown)
def build_two_stage_profile_report_md(
    alpha_econ: pd.DataFrame,
    beta_econ: pd.DataFrame,
    econ_features_all: list,
    alpha_socio: pd.DataFrame,
    beta_socio: pd.DataFrame,
    socio_features_all: list,
    suff_threshold: float = 0.7,
    round_beta: int = 4,
    out_path: str = REPORT_PATH
):
    idx_ref = alpha_econ.index
    if not (idx_ref.equals(beta_econ.index) and idx_ref.equals(alpha_socio.index) and idx_ref.equals(beta_socio.index)):
        raise ValueError("Indexmengen (Profile) von alpha_econ/beta_econ/alpha_socio/beta_socio müssen identisch sein.")

    econ_feats = [f for f in econ_features_all if f in alpha_econ.columns and f in beta_econ.columns]
    socio_feats = [f for f in socio_features_all if f in alpha_socio.columns and f in beta_socio.columns]

    md = []
    md.append("# Profilbericht (zweistufig): α/β je Profil – Ökonomisch vs. Soziodemographisch\n")
    md.append(f"- **Anzahl Profile:** {len(idx_ref)}")
    md.append(f"- **Hinreichend-Schwelle:** β ≥ {suff_threshold}")
    md.append(f"- **Ökonomische Features:** {', '.join(econ_feats) if econ_feats else '(keine)'}")
    md.append(f"- **Soziodemographische Features:** {', '.join(socio_feats) if socio_feats else '(keine)'}\n")

    # Überblick je Profil
    overview_rows = []
    for pid in idx_ref:
        row = {"profile_index": pid}
        row["econ_alpha_cnt"] = int(alpha_econ.loc[pid, econ_feats].sum()) if econ_feats else 0
        row["econ_hinr_cnt"]  = int((beta_econ.loc[pid, econ_feats] >= suff_threshold).sum()) if econ_feats else 0
        row["socio_alpha_cnt"] = int(alpha_socio.loc[pid, socio_feats].sum()) if socio_feats else 0
        row["socio_hinr_cnt"]  = int((beta_socio.loc[pid, socio_feats] >= suff_threshold).sum()) if socio_feats else 0
        overview_rows.append(row)
    overview_df = pd.DataFrame(overview_rows).set_index("profile_index")
    md.append("## Überblick (Anzahl notwendiger / hinreichender Features je Stufe)\n")
    md.append(overview_df.to_markdown())
    md.append("\n---\n")

    # Details je Profil
    md.append("## Details je Profil\n")
    for pid in idx_ref:
        display_id = pid + 1  # 1-basierte Anzeige
        md.append(f"\n### Profil {display_id}\n")
        if econ_feats:
            df_e = pd.DataFrame({
                "Feature": econ_feats,
                "alpha_notwendig": alpha_econ.loc[pid, econ_feats].astype(int).values,
                "beta": np.round(beta_econ.loc[pid, econ_feats].astype(float).values, round_beta),
            })
            df_e[f"hinreichend (β≥{suff_threshold:.2f})"] = (df_e["beta"] >= suff_threshold).astype(int)
            md.append("\n**Ökonomische Features**\n")
            md.append(df_e.to_markdown(index=False))

        if socio_feats:
            df_s = pd.DataFrame({
                "Feature": socio_feats,
                "alpha_notwendig": alpha_socio.loc[pid, socio_feats].astype(int).values,
                "beta": np.round(beta_socio.loc[pid, socio_feats].astype(float).values, round_beta),
            })
            df_s[f"hinreichend (β≥{suff_threshold:.2f})"] = (df_s["beta"] >= suff_threshold).astype(int)
            md.append("\n**Soziodemographische/sonstige Features**\n")
            md.append(df_s.to_markdown(index=False))

        md.append("\n---")

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(md))


**Schritt 10** Ausführung

-  Hier wird die α/β-Evaluation getrennt für ökonomische und soziodemographische Feature-Sets ausgeführt.

- Dann wird der finale Bericht als Markdown erzeugt und unter REPORT_PATH gespeichert 

In [21]:
# Ausführung + Bericht
alpha_econ, alpha_econ_mean = run_alpha_for_profiles(
    model, preprocessor, df_all=X_all, profiles_df=profiles_df,
    features_numeric=econ_num, features_categorical=econ_cat,
    threshold=0.5, q_lo=0.05, q_hi=0.95, n_steps=9
)
beta_econ,  beta_econ_mean  = run_beta_for_profiles(
    model, preprocessor, df_all=X_all, profiles_df=profiles_df,
    features_numeric=econ_num, features_categorical=econ_cat,
    threshold=0.5, n_samples=2000, random_state=42
)

alpha_socio, alpha_socio_mean = run_alpha_for_profiles(
    model, preprocessor, df_all=X_all, profiles_df=profiles_df,
    features_numeric=socio_num, features_categorical=socio_cat,
    threshold=0.5, q_lo=0.05, q_hi=0.95, n_steps=9
)
beta_socio,  beta_socio_mean  = run_beta_for_profiles(
    model, preprocessor, df_all=X_all, profiles_df=profiles_df,
    features_numeric=socio_num, features_categorical=socio_cat,
    threshold=0.5, n_samples=2000, random_state=4242
)

build_two_stage_profile_report_md(
    alpha_econ=alpha_econ, beta_econ=beta_econ, econ_features_all=econ_num+econ_cat,
    alpha_socio=alpha_socio, beta_socio=beta_socio, socio_features_all=socio_num+socio_cat,
    suff_threshold=0.7, round_beta=4, out_path=REPORT_PATH
)

print(f"💾 Zweistufiger Profilbericht gespeichert unter: {REPORT_PATH}")


💾 Zweistufiger Profilbericht gespeichert unter: C:\Users\JonasNiehus\Documents\Masterarbeit\Evaluation\Masterarbeit_Guetekriterien-sichere-und-interpetierbare-Hochrisiko-KI-Systeme\results\profilbericht_alpha_beta_twostage.md
